Bayesian Optimization & NN Opt
---

In [1]:
from bayes_opt import BayesianOptimization
from nnopt.nnopt import Optimizer
import time
import numpy as np
import gym


In [2]:
ll = gym.make("MountainCarContinuous-v0")
print(ll.action_space)
print(ll.observation_space)

def black_box_functionA(a, b):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    transform = np.array([[a, b]])
    
    sc = 0
    for _ in range(4):
        s = ll.reset()
        done = False
        while not done:
            #ll.render()
            s, r, done, _ = ll.step(transform @ s)
            sc += r

    return sc / 4

def black_box_functionB(a, b):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -a ** 2 - (b - 1) ** 2 + 1

#print(black_box_function(-221.59328, 28.267517))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Box(1,)
Box(2,)


/usr/local/lib/python3.6/site-packages/gym/envs/registration.py:14: DeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
Nopt = Optimizer(black_box_functionB, N=2, surrogate_hidden_layer=1000, model_hidden_layer=400, memory_hidden_layer=400,
                 Rn=[(-10, 10), (-10, 10)]) #(-10, 10), (-10, 10), (-10, 10), (-10, 10)])

Bopt = BayesianOptimization(
    f=black_box_functionB,
    pbounds={'a': (-10, 10), 'b': (-10, 10)#, 'c': (-10, 10), 
             #'d': (-10, 10), 'e': (-10, 10), 'f': (-10, 10)},
            },
    random_state=np.random.randint(10),
)

Bayesian
---

In [4]:
timestamp = time.time()
Bopt.maximize(
    init_points=40,
    n_iter=5,
)

print("Bayesian took: {}".format(time.time() - timestamp))

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        | -86.25    |  9.341    |  0.9446   |
|  2        | -99.24    |  9.454    |  4.296    |
|  3        | -59.24    |  3.955    | -5.678    |
|  4        | -208.0    |  9.525    | -9.875    |
|  5        | -28.72    | -4.94     | -1.304    |
|  6        | -79.87    |  5.588    | -6.046    |
|  7        | -126.8    |  7.26     |  9.668    |
|  8        | -45.1     | -6.723    |  1.947    |
|  9        | -106.1    | -9.82     | -2.269    |
|  10       | -148.3    | -9.117    |  9.133    |
|  11       | -64.3     | -1.277    |  8.98     |
|  12       | -71.8     |  5.726    |  7.326    |
|  13       | -132.0    | -6.537    | -8.501    |
|  14       | -61.44    |  2.015    | -6.641    |
|  15       | -28.8     |  4.668    | -1.831    |
|  16       | -59.4     |  0.5582   |  8.751    |
|  17       | -77.27    |  0.4339   | -7.836    |
|  18       | -45.73    | -6.836    |  0.9041   |


In [ ]:
timestamp = time.time()
best, target = Nopt.run(random=50, optimization=10, fitting=10000, verbose=True)

print("Best", best, "target", target)
print(black_box_functionB(*best))
print("NNopt took:", time.time() - timestamp)

sample [-7.9506826  3.7805414] target -69.94476503144142
sample [8.921061  8.7759495] target -139.05071183999098
sample [-9.064784    0.49090862] target -81.42948390331185
sample [-2.6754522  -0.28281212] target -7.80365157909273
sample [-8.649967 -7.07283 ] target -138.99251989160257
sample [ 0.26113987 -6.4866543 ] target -55.11818636398402
sample [ 6.9432354 -9.17829  ] target -150.8061125803988
sample [-8.628116  -0.6577587] target -76.19254368841666
sample [ 1.1841869 -0.7453203] target -3.4484417178873628
sample [ 8.977568  -7.9508066] target -159.71366042655882
sample [2.1747952 7.562664 ] target -46.79829314443123
sample [3.8204384 3.562725 ] target -20.163309223396936
sample [-2.3197556  3.2399035] target -9.398433296614712
sample [5.474572  2.1385307] target -30.26719279855115
sample [-7.9304385  7.303171 ] target -101.62182174107897
sample [ 1.2476997 -6.2512684] target -53.13764785307649
sample [-6.2219334  7.4100323] target -78.80096852928546
sample [4.150319  0.6056547] t